# Everything from Part2 of the assignment

In [1]:
import geocoder
import pandas as pd
data = pd.read_csv("neighborhoods.csv")
postcode = pd.read_csv("Geospatial_Coordinates.csv")
lat = []
lon = []
for i in range(len(data['PostalCode'])):
    for j in range(len(postcode['Postal Code'])):
        if data.loc[i,'PostalCode'] == postcode.loc[j,'Postal Code']:
            lat.append(postcode.loc[j,'Latitude'])
            lon.append(postcode.loc[j,'Longitude'])
data['Latitude'] = lat
data['Longitude'] = lon
data = data.drop('Unnamed: 0',axis = 1)

## Checking the dadaframe before going forward

In [2]:
data.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor,Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill,Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District,Ryerson",43.657162,-79.378937


## Everything looks good so we can start clustering now.

# 1. Import the necessary modules

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.....')

Libraries imported.....


# 2. Use geopy library to get the latitude and longitude values of Toronto City.

In [4]:
address = 'Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto are 43.653963, -79.387207.


# 3. Create a map of Toronto with neighborhoods superimposed on top.

In [5]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood,code in zip(data['Latitude'], data['Longitude'], data['Borough'], data['Neighborhood'], data['PostalCode']):
    label = '{},Borough : {}, Postal-Code :{}'.format(neighborhood, borough, code)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# 4. Now that it is done we can start using the Foursquare API 

In [10]:
CLIENT_ID = 'J5D0K1WE2SCLGSCHLQQNNJSCW0VTWEPQ3SOCO5FTQ00U3NMU' # your Foursquare ID
CLIENT_SECRET = 'B3S2Q3MWGMBYO4ASTYTTGDIZMZRISSTXZF1WTPZBGJJVZX14' # your Foursquare Secret
VERSION = '20180604'
#CLIENT_ID = '*********' # your Foursquare ID
#CLIENT_SECRET = '*****' # your Foursquare Secret
#VERSION = '***********' # Version                  #deleted because contains sensitive data
LIMIT = 100

# 5. Define the function to extract the category from the venues json data

In [11]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# 6. Time to explore the neighborhoods in Toronto

In [14]:
# Here we shall restrict our exploration within 500 meters radius of the neiughborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
#    count = 0
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#        count+=1
#        print(count,name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# 7. Create the Dataframe that has all the data about the Neighborhoods and the venues within the 500 metres radius of the Neighborhood.

In [15]:
toronto_venues = getNearbyVenues(names = data['Neighborhood'], 
                                   latitudes = data['Latitude'], 
                                   longitudes = data['Longitude'])

In [16]:
toronto_venues.head(15)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Parkwoods,43.753259,-79.329656,Bella Vita Catering & Private Chef Service,43.756651,-79.331524,BBQ Joint
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
5,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
6,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
7,Victoria Village,43.725882,-79.315572,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection
8,"Regent Park,Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
9,"Regent Park,Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop


#### Let's have a look at the number of venues we got

In [ ]:
print("We have {} venues near all {} neighborhoods in Toronto".format(toronto_venues.shape[0],len(toronto_venues['Neighborhood'].unique())))

An important observation to make here is that although we have processed all the 103 neighborhoods we have data for 100 neighborhoods which means that foursquare has no data on venues within 500 meters radius of 3 neighborhoods.

#### Let's figure out which Neighborhoods are missing

In [ ]:
for i in data['Neighborhood']:
    if i not in toronto_venues['Neighborhood'].unique():
        print(i)

### Now that we have the missing Neighborhoods let's take one of them and check our observation.    

Let's take ***Islington Avenue*** as the target Neighborhood and check the co-ordinates (43.667856,-79.532242) in **Foursquare**.

In [ ]:
url = 'http://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&limit={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION,43.667856,-79.532242,100,500)

In [ ]:
results = requests.get(url).json()
results

### The warning looks worth noticing since the 'items' is empty.

In [ ]:
results['response']['warning']['text']

# 8. Let's expand our search area to 1.5 km.

In [17]:
toronto_venues = getNearbyVenues(names = data['Neighborhood'], 
                                   latitudes = data['Latitude'], 
                                   longitudes = data['Longitude'], radius=1500)

In [18]:
toronto_venues.head(15)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods,43.753259,-79.329656,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course
3,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
4,Parkwoods,43.753259,-79.329656,LCBO,43.757774,-79.314257,Liquor Store
5,Parkwoods,43.753259,-79.329656,A&W Canada,43.760643,-79.326865,Fast Food Restaurant
6,Parkwoods,43.753259,-79.329656,Tim Hortons,43.752814,-79.314067,Coffee Shop
7,Parkwoods,43.753259,-79.329656,Bruno's valu-mart,43.746086,-79.324978,Grocery Store
8,Parkwoods,43.753259,-79.329656,Shoppers Drug Mart,43.745303,-79.325249,Pharmacy
9,Parkwoods,43.753259,-79.329656,Food Basics,43.760865,-79.326015,Supermarket


#### Let's have a look at the number of venues we got this time

In [19]:
print("We have {} venues near all {} neighborhoods in Toronto".format(toronto_venues.shape[0],len(toronto_venues['Neighborhood'].unique())))

We have 6760 venues near all 103 neighborhoods in Toronto


### So we have got all the 103 Neighborhoods this time.

### Now let's check the number of venues per Neighborhood

In [ ]:
# Number of venues per neighbourhood
toronto_venues.groupby('Neighborhood')['Venue'].count()

# 9. Analyze Each Neighborhood

In [20]:
print("Number of unique categories are : ",len(toronto_venues['Venue Category'].unique()))

Number of unique categories are :  340


In [28]:
toronto_hotels = toronto_venues[toronto_venues['Venue Category']=='Hotel']

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood_name'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
print(toronto_onehot.shape[1]) # checking whether all the 334 categories are included and total number of columns should be 335
toronto_onehot.head()

### One problem that we faced during the craetion of this dataframe is that one of the 334 categories of the venues is called 'Neighborhood'. So we have to careful about giving a new name to the column. Here we have given the name 'Neighborhood_name'. Also the total number of column is 335 as expected.

In [ ]:
#the shape of the dataframe is :
toronto_onehot.shape

This shape looks perfect!!

# 10. Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
toronto_grouped=toronto_onehot.groupby('Neighborhood_name').mean().reset_index()
toronto_grouped

### So let's take a neighbourhood and print it's top 5 most common venues.

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood_name']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood_name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

# 11.  Let's put top ten venues of each neighborhood into a **Pandas** dataframe

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood_name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood_name'] = toronto_grouped['Neighborhood_name']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

# 12. Cluster Neighborhood

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood_name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(len(kmeans.labels_))
kmeans.labels_

In [ ]:
toronto_merged = data

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood_name'), on='Neighborhood')
toronto_merged

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# 13. Examine the clusters

### Cluster 1

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

### Cluster 2

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

### Cluster 3

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

### Cluster 4

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

### Cluster 5

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]